In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import acquire
print("Scucess")

Scucess


# Pre Zillow
This notebook is used to prepare the zillow_wrangle.py function by performing basic operations to see things like value counts on columns, detect outliers, etc.

In [2]:
help(acquire)

Help on module acquire:

NAME
    acquire

FUNCTIONS
    columns_data(df)
        Takes in a data frame and returns the number of rows and percentage of rows missing per column
    
    get_connection(db, user='darden_1027', host='157.230.209.171', password='nIymKg6jJmIPe6gQxik57EkY3Fl9kmvl')
        Returns a formatted url with login credentials to access data on a SQL database.
    
    load_zillow_data()
        This function acquires the zillow dataset from a SQL Database.
        It returns the zillow dataset as a Pandas DataFrame.
        
        A local copy will be created as a csv file in the current directory for future use.
    
    row_data(df)
        Takes in a data frame and returns the range of columns with missing data, 
        what precentage of missing columns that represents,
        and how many rows have that number of missing columns
    
    summarize(df)
        Prints the basic summary statistics of a data frame

FILE
    c:\users\anthony\documents\data_scie

In [3]:
df = acquire.load_zillow_data()
df.shape

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(77381, 70)

In [4]:
acquire.row_data(df)

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.328571
1,24,13,0.342857
2,25,24,0.357143
3,26,65,0.371429
4,27,316,0.385714
5,28,455,0.400000
6,29,5270,0.414286
7,30,3455,0.428571
8,31,9891,0.442857
9,32,12579,0.457143


In [5]:
acquire.columns_data(df)

,columns,num_rows_missing,pct_rows_missing
0,buildingclasstypeid,77366,0.999806
1,typeconstructiontypeid,77159,0.997131
2,storytypeid,77331,0.999354
3,propertylandusetypeid,0,0.000000
4,heatingorsystemtypeid,27941,0.361083
...,...,...,...
65,heatingorsystemdesc,27941,0.361083
66,propertylandusedesc,0,0.000000
67,storydesc,77331,0.999354
68,typeconstructiondesc,77159,0.997131


In [6]:
df.poolcnt.value_counts()

1.0    16137
Name: poolcnt, dtype: int64

In [7]:
df.basementsqft.isnull().sum()


77331

In [8]:
df.storydesc.unique()

array([nan, 'Basement'], dtype=object)

In [9]:
df.heatingorsystemdesc.unique()

array([nan, 'Central', 'Floor/Wall', 'Forced air', 'Yes', 'None',
       'Radiant', 'Solar', 'Baseboard', 'Gravity', 'Heat Pump'],
      dtype=object)

In [10]:
print(df.fireplacecnt.isnull().sum())
print(df.fireplaceflag.isnull().sum())

69105
77209


In [11]:
((df['fireplaceflag'] == 1) == (df.fireplacecnt.isnull())).mean() * 100

10.917408666210052

In [12]:
df.hashottuborspa.unique()

array([nan,  1.])

In [13]:
df['propertylandusedesc'].loc[df['unitcnt'] == 1].value_counts()

Single Family Residential                   33734
Condominium                                 11494
Planned Unit Development                     1937
Mobile Home                                    53
Cooperative                                    29
Duplex (2 Units, Any Combination)              20
Residential General                            10
Commercial/Office/Residential Mixed Used        8
Townhouse                                       6
Triplex (3 Units, Any Combination)              2
Name: propertylandusedesc, dtype: int64

In [14]:
df['fips'].loc[df['unitcnt'] == 1].value_counts()

6037.0    47282
6059.0       10
6111.0        1
Name: fips, dtype: int64

In [15]:
def prep_zillow():
    '''
    Loads zillow data and then fliters it where propertylandusetypeid is 260, 261, 262, 263, 264, 265, 268, 275, 276, or 279.
    Drops unnecessary id columns and any duplicated columns.    
    Creates a has_ool column where poolcnt = 1.
    Creates a has_basement column where basementsqft > 0.
    Replaces nan values with 'none' for heatingorsystemdesc
    Replaces nan values with 0 for hashottuborspa. This was done because the only values were nan or 1
    Drops calculatedbathnbr, fullbathcnt,  columns 
    Drops columns where the percentage of missing data is over 40%
    Drop rows where the percentage of missing data is over 50%
    '''
    df = acquire.load_zillow_data()
    df = df[df.propertylandusetypeid.isin([260, 261, 262, 263, 264, 265, 268, 275, 276, 279])]
    df.drop(columns = ['buildingclasstypeid', 'typeconstructiontypeid', 'storytypeid',
                   'propertylandusetypeid', 'heatingorsystemtypeid', 'architecturalstyletypeid',
                   'airconditioningtypeid', 'id', 'parcelid' , 'unitcnt', 'propertyzoningdesc', 
                   'finishedsquarefeet12', 'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt'], inplace = True)
    df.drop(columns = [c for c in df.columns if c.endswith('.1')], inplace=True)
    df['has_pool'] = df['poolcnt'] == 1
    df['has_basement'] = df['basementsqft'] > 0
    df.heatingorsystemdesc.fillna('none', inplace = True)
    df.hashottuborspa.fillna(0, inplace = True)
    df.replace({True:1, False:0}, inplace = True)
    df = handle_missing_values(df, .6, .5)
    return df

In [16]:
def handle_missing_values(df, prop_required_column=.5, prop_required_row=.5):
    '''
    This function takes in a DataFrame (df), a minium for prop_required_column [0:1] with deafult of .5, 
    and a minimum value [0:1] for prop_required_row with a default of .5.
    
    It will first drop columns who's missing data is less than prop_required_column value.
    It will then drop the rows who's missing data is lower than the prop_required_row.
    '''
    columns_to_keep = []
    columns = df.columns
    for col in columns:   
        if df[col].notnull().sum()/df.shape[0] >= prop_required_column:
            columns_to_keep.append(col)
    df = df[columns_to_keep]
    df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row
    df = df.loc[df['row_to_keep'] == True]
    df.drop(columns= ['row_to_keep'], inplace = True)
    return df

In [17]:
zillow = prep_zillow()
zillow.shape

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-16-38dc43d33f77>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row


(52828, 28)

In [18]:
zillow.isnull().sum().sort_values(ascending=False)

buildingqualitytypeid           19106
regionidcity                     1036
lotsizesquarefeet                 408
yearbuilt                         159
calculatedfinishedsquarefeet      127
censustractandblock               125
structuretaxvaluedollarcnt         93
regionidzip                        27
taxamount                           4
landtaxvaluedollarcnt               2
taxvaluedollarcnt                   1
propertycountylandusecode           0
hashottuborspa                      0
fips                                0
latitude                            0
longitude                           0
bedroomcnt                          0
has_basement                        0
rawcensustractandblock              0
regionidcounty                      0
has_pool                            0
roomcnt                             0
assessmentyear                      0
transactiondate                     0
logerror                            0
heatingorsystemdesc                 0
propertyland

In [19]:
19106/zillow.shape[0]

0.361664268948285

In [20]:
zillow.buildingqualitytypeid.unique()

array([nan,  8.,  9.,  5.,  4.,  6.,  7., 10., 11.,  3., 12.,  2.,  1.])

In [21]:
zillow.heatingorsystemdesc.unique()

array(['none', 'Central', 'Floor/Wall', 'Forced air', 'Radiant', 'Solar',
       'None', 'Yes', 'Gravity', 'Baseboard'], dtype=object)

In [22]:
print(zillow['buildingqualitytypeid'].describe())
print(zillow['buildingqualitytypeid'].median())
print(zillow['buildingqualitytypeid'].mode())
print(zillow['buildingqualitytypeid'].mean())


count    33722.000000
mean         6.266947
std          1.716054
min          1.000000
25%          5.000000
50%          6.000000
75%          8.000000
max         12.000000
Name: buildingqualitytypeid, dtype: float64
6.0
0    6.0
dtype: float64
6.266947393393037


In [23]:
zillow['fips'].value_counts()

6037.0    33921
6059.0    14121
6111.0     4786
Name: fips, dtype: int64

In [24]:
zillow['regionidcity'].value_counts()

12447.0    11439
5534.0      1808
40227.0     1499
46298.0     1426
16764.0     1085
           ...  
32927.0        3
31134.0        2
10815.0        1
36078.0        1
21395.0        1
Name: regionidcity, Length: 175, dtype: int64

In [25]:
zillow['regionidcounty'].value_counts()

3101.0    33921
1286.0    14121
2061.0     4786
Name: regionidcounty, dtype: int64

In [26]:
df.regionidneighborhood.isnull().sum()

46491

In [27]:
zillow[['regionidcounty', 'regionidzip']].loc[zillow['regionidzip'].isnull()]

,regionidcounty,regionidzip
4161,3101.0,NaN
9732,3101.0,NaN
10063,3101.0,NaN
13128,3101.0,NaN
13386,3101.0,NaN
18423,2061.0,NaN
26735,1286.0,NaN
30920,2061.0,NaN
34551,2061.0,NaN
34894,2061.0,NaN


In [28]:
df.propertylandusetypeid.value_counts()

261.0    52320
266.0    19294
246.0     2009
269.0     1944
248.0      727
247.0      535
265.0      333
263.0       74
275.0       58
260.0       37
267.0       29
31.0        15
264.0        6
Name: propertylandusetypeid, dtype: int64

Imputing missing region city id

I have a feelign that regioncity id is a important feature to have and, rather than gettin rid of over 1k additional rows, I'm going to figure out a way to approximate which city id belongs to which zips code, which is not missing nearly as many values. To do this I will find the zip codes associated with missing cityids, excluding nan, and then find the mode of the city id assoicated with those values. This will be used to impute the values manually.

In [29]:
zips = zillow[['regionidcity', 'censustractandblock','regionidzip']].loc[zillow['regionidcity'].isnull()].regionidzip.unique()
zips = [96186., 96956., 96337., 96989., 96993., 96395., 97023., 96377.,
       95983., 96368., 96174., 96292., 96494., 96403., 96244., 96373.,
       96295., 97027., 96962., 96346., 96352., 96026., 96040., 96507.,
       96969., 96982., 96110., 97008., 96522., 96273., 96492.,
       96488., 96169., 96270., 96203., 97081., 96506., 97026., 96973.,
       97043., 97003., 96204., 96975., 97005., 96247., 96974., 97048.,
       96342., 96105., 96990., 96387., 96212., 97065., 96225., 97047.,
       96489., 96374., 96221., 96959., 96398., 96296., 97018., 96171.,
       96229., 96505., 97083., 96126., 96049., 96284., 96242., 96044.,
       97016., 96378.]

In [89]:
findcity = zillow[['regionidcity', 'regionidzip']].loc[(zillow['regionidzip'].isin(zips))]
pairs = dict(findcity.groupby('regionidzip')['regionidcity'].agg(pd.Series.mode))
pairs_dict = pd.DataFrame(findcity.groupby('regionidzip')['regionidcity'].agg(pd.Series.mode)).reset_index()
pairs

{95983.0: 12447.0,
 96026.0: 12447.0,
 96040.0: 12447.0,
 96044.0: 17882.0,
 96049.0: 12447.0,
 96105.0: 12447.0,
 96110.0: 39076.0,
 96126.0: 56780.0,
 96169.0: 14634.0,
 96171.0: 14634.0,
 96174.0: 396556.0,
 96186.0: 18874.0,
 96203.0: 42967.0,
 96204.0: 30267.0,
 96212.0: 12292.0,
 96221.0: 12447.0,
 96225.0: 54352.0,
 96229.0: 10723.0,
 96242.0: 46298.0,
 96244.0: 46298.0,
 96247.0: 46298.0,
 96270.0: 24435.0,
 96273.0: 32923.0,
 96284.0: 12447.0,
 96292.0: 47019.0,
 96295.0: 47019.0,
 96296.0: 54299.0,
 96337.0: 37688.0,
 96342.0: 12447.0,
 96346.0: 12447.0,
 96352.0: 54311.0,
 96368.0: 12447.0,
 96373.0: 54311.0,
 96374.0: 54311.0,
 96377.0: 54311.0,
 96378.0: 54311.0,
 96387.0: 12447.0,
 96395.0: array([], dtype=float64),
 96398.0: 10734.0,
 96403.0: 54311.0,
 96488.0: 39306.0,
 96489.0: 45602.0,
 96492.0: 22827.0,
 96494.0: 39308.0,
 96505.0: 24384.0,
 96506.0: 20008.0,
 96507.0: 20008.0,
 96522.0: 34636.0,
 96956.0: 13693.0,
 96959.0: 38032.0,
 96962.0: 32380.0,
 96969.0: 528

In [37]:
pairs_dict

,regionidzip,regionidcity
0,95983.0,12447
1,96026.0,12447
2,96040.0,12447
3,96044.0,17882
4,96049.0,12447
...,...,...
68,97047.0,24832
69,97048.0,24832
70,97065.0,33252
71,97081.0,34543


Additional columns to drop:
* unitcnt 
* propertyzoningdesc
* finishedsquarefeet12
* calculatedbathnbr
* finishedsquarefeet12
* fullbathcnt

Drop rows if full
* regionidzip - Due to having so few zips codes but properties exist across multiple counties
* 


In [99]:
test = zillow.merge(pairs_dict, left_on='regionidzip', right_on='regionidzip')
test.isnull().sum()

bathroomcnt                        0
bedroomcnt                         0
buildingqualitytypeid           5497
calculatedfinishedsquarefeet      26
fips                               0
hashottuborspa                     0
latitude                           0
longitude                          0
lotsizesquarefeet                134
propertycountylandusecode          0
rawcensustractandblock             0
regionidcity_x                  1012
regionidcounty                     0
regionidzip                        0
roomcnt                            0
yearbuilt                         31
structuretaxvaluedollarcnt         5
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          0
censustractandblock               25
transactiondate                    0
logerror                           0
heatingorsystemdesc                0
propertylandusedesc                0
has_pool                           0
h

In [98]:
test['regionidcity_x'].fillna(test['regionidcity_y'], inplace=True)
test.isnull().sum()

bathroomcnt                        0
bedroomcnt                         0
buildingqualitytypeid           5497
calculatedfinishedsquarefeet      26
fips                               0
hashottuborspa                     0
latitude                           0
longitude                          0
lotsizesquarefeet                134
propertycountylandusecode          0
rawcensustractandblock             0
regionidcity_x                     0
regionidcounty                     0
regionidzip                        0
roomcnt                            0
yearbuilt                         31
structuretaxvaluedollarcnt         5
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          0
censustractandblock               25
transactiondate                    0
logerror                           0
heatingorsystemdesc                0
propertylandusedesc                0
has_pool                           0
h

In [41]:
zillow.isnull().sum()

bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           19106
calculatedfinishedsquarefeet      127
fips                                0
hashottuborspa                      0
latitude                            0
longitude                           0
lotsizesquarefeet                 408
propertycountylandusecode           0
rawcensustractandblock              0
regionidcity                     1036
regionidcounty                      0
regionidzip                        27
roomcnt                             0
yearbuilt                         159
structuretaxvaluedollarcnt         93
taxvaluedollarcnt                   1
assessmentyear                      0
landtaxvaluedollarcnt               2
taxamount                           4
censustractandblock               125
transactiondate                     0
logerror                            0
heatingorsystemdesc                 0
propertylandusedesc                 0
has_pool    